# 🔍 Document Question Answering with Local LLMs and Embeddings

This notebook demonstrates how to build a **local, privacy-friendly document question answering (QA) system** using [LangChain](https://www.langchain.com/), [Ollama](https://ollama.com/), and [Chroma](https://www.trychroma.com/). It loads a PDF document, splits it into chunks, generates embeddings locally using the `nomic-embed-text` model, and stores them in a persistent Chroma vector database. A retriever fetches relevant chunks in response to a query, and a locally running LLM (e.g., `gemma3:1b`) is used to generate an accurate answer with citations. This workflow avoids cloud APIs, making it ideal for secure and offline environments.


In [7]:
# === Imports ===
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.chains import RetrievalQA

# Import the necessary modules for working with documents, splitting text,
# creating embeddings, managing vector storage, and performing question-answering.

# === Load a PDF file ===
loader = PyPDFLoader("./data/Agentic_Workflows.pdf")  # Load the PDF file specified in the path
docs = loader.load()  # Read the content of the PDF and prepare it for processing

# === Split the document into manageable chunks ===
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Set the maximum size of each chunk to 1000 characters
    chunk_overlap=200  # Ensure 200 characters overlap between chunks for context
)
splits = text_splitter.split_documents(docs)  # Split the loaded document into smaller chunks

# === Use Ollama's nomic-embed-text to create embeddings ===
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")  # Convert text into numerical representations (embeddings)

# === Create a Chroma vectorstore with persistence ===
persist_directory = './data/db/chroma/'  # Set the directory to save the vectorstore for later use

# Store the embeddings and text chunks in a Chroma vectorstore, enabling persistent storage
vectorstore = Chroma.from_documents(
    documents=splits,  # The document chunks to store
    embedding=embeddings,  # The embedding function to use for processing text
    persist_directory=persist_directory  # Directory for storing data persistently
)

# === Load the persisted Chroma vectorstore ===
vector_store = Chroma(
    persist_directory=persist_directory,  # Load the saved vectorstore from the specified directory
    embedding_function=embeddings  # Use the same embedding function as before
)

# === Set up a retriever from the vectorstore ===
retriever = vector_store.as_retriever(search_kwargs={"k": 2})  # Configure the retriever to return the top 2 most relevant results

# === Use Ollama's local chat model ===
llm = ChatOllama(model="gemma3:1b", temperature=0.0)  # Initialize a chat model with deterministic responses (temperature set to 0.0)

# === Set up a RetrievalQA chain for Q&A over documents ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # Use the language model for generating answers
    chain_type="stuff",  # Specify the type of chain to process retrieved document chunks
    retriever=retriever,  # The retriever to fetch relevant document parts
    verbose=True,  # Enable detailed logging for debugging
    return_source_documents=True  # Include source documents in the response for transparency
)

# === Pretty-printing helper to show results and sources ===
def process_llm_response(llm_response):
    # Print the main result (answer) from the language model
    print(llm_response['result'])
    print('\n\nSources:')
    # Print the sources used to generate the response
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

# === Ask a question! ===
query = "Talk to me about the two main challenges of agentic workflows"  # Define a question to query the chain
llm_response = qa_chain.invoke(query)  # Send the question to the RetrievalQA chain and get the response

# === Display the result ===
process_llm_response(llm_response)  # Format and print the answer along with its sources



> Entering new RetrievalQA chain...

> Finished chain.
Okay, let’s break down the two main challenges identified in the text regarding agentic workflows – and then I’ll expand on it.

Based on the provided context, the two main challenges are:

1.  **Ambiguity and Transient Interactions:** The text explicitly states that the interactions are “brief and transient.” This means that the collaboration between humans and AI agents is often short-lived.  The goal is to achieve complex goals, but the rapid, fragmented nature of these interactions makes it difficult to maintain a sustained engagement with users.  It’s hard to build a deep understanding of the user’s needs and intentions in a way that allows for continuous refinement.

2.  **Limited Understanding of AI Functionalities:** The text highlights a lack of understanding of AI functionalities.  The AI agents are essentially acting as “stand-ins” for users, and the system isn’t yet equipped to fully grasp the nuances of how AI can tr